# Import

## Libraries

In [1]:
# Warning control
import warnings
import os
import yaml
from pathlib import Path


from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool, DirectoryReadTool

from pydantic import BaseModel, Field
from typing import List, Optional
from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

## Conversation Transcript

Output from `pdf_to_md.ipynb`:

In [2]:
transcript_content = Path("data/conversation_01.txt").read_text()
print(transcript_content[:500])

CSR: Good morning! Thank you for calling ShiokAh Insurance. My name is Sarah. How may I assist you today?

Customer: Hi Sarah! I want to check about travel insurance leh. Planning to go Japan next month.

CSR: I'll be happy to help you with that. Before we look into the options, could you share some information with me? May I know your age?

Customer: I'm 62 this year.

CSR: Thank you for sharing. Since you're above 60, we'll need to note this for coverage considerations. Do you have any pre-exi


## Tasks & Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [4]:
print(
    "agents_config contains:",
    ", ".join(agents_config.keys()),
    "\ntasks_config contains:",
    ", ".join(tasks_config.keys()),
)

agents_config contains: conversation_transcript_agent, underwriting_report_agent 
tasks_config contains: process_transcript_task, generate_underwriting_report_task


## Tools (Agent Usage)

Got `FileReadTool()` to work, thanks to this [article](https://lablab.ai/t/crewai-multi-agent-system)!

In [5]:
directory_read_tool = DirectoryReadTool(directory="./data")
file_read_tool = FileReadTool()


# read_conversation = FileReadTool(file_path="./data/conversation_01.txt")
# read_income = FileReadTool(file_path="./data/income_travel_insurance_2024.txt")
# read_ge = FileReadTool(file_path="data/ge_travel_insurance_2024.txt")

# Pydantic Classes

In [6]:
class CustomerInfo(BaseModel):
    age: int
    medical_conditions: Optional[List[str]] = Field(default_factory=list)
    destination: str
    coverage_factors: List[str] = Field(default_factory=list)
    policy_customization_factors: List[str] = Field(default_factory=list)


class CustomerInfoList(BaseModel):
    customers: List[CustomerInfo] = Field(default_factory=list)

# Create Crew, Agents and Tasks

[Agent attributes](https://docs.crewai.com/concepts/agents) for reference.  
Tools can be assigned when specifying Agent:

In [7]:
conversation_transcript_agent = Agent(
    config=agents_config["conversation_transcript_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

underwriting_report_agent = Agent(
    config=agents_config["underwriting_report_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

[Task attributes](https://docs.crewai.com/concepts/tasks) for reference.

In [8]:
process_transcript_task = Task(
    config=tasks_config["process_transcript_task"],
    agent=conversation_transcript_agent,
    output_pydantic=CustomerInfoList,
)

generate_underwriting_report_task = Task(
    config=tasks_config["generate_underwriting_report_task"],
    context=[process_transcript_task],
    agent=underwriting_report_agent,
    async_execution=True,
)

[Crew attributes](https://docs.crewai.com/concepts/crews) for reference.  

The `process` parameter controls how tasks are executed by the crew. The options include:

* Sequential (Default): Tasks are performed one after the other in a specific order.
* Hierarchical: One agent acts as a manager, delegating tasks to other agents while maintaining an overarching memory of the tasks.
* Parallel: Tasks are executed concurrently, allowing multiple tasks to run at the same time.


In [9]:
recsys_crew = Crew(
    agents=[conversation_transcript_agent, underwriting_report_agent],
    tasks=[
        process_transcript_task,
        generate_underwriting_report_task,
    ],
    verbose=True,
    process="sequential",
)

In [10]:
result = recsys_crew.kickoff()
display(Markdown(result.raw))


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Task: This task involves analyzing a conversation transcript between a potential customer interested in travel insurance and a customer service representative from ShiokAh Insurance. The goal is to extract key customer details from the transcript, such as age, medical conditions, destination, coverage preferences, and customization requests. This extracted information will be organized into a Python list and used by an underwriter to determine policy suitability.


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: Thought: To begin the task appropriately, I need to locate the file {{conversation_01.txt}} in the directory and then read its content.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./data/conversation_01.txt
- ./data/ge_travel_insurance_2024.txt
- ./data/income_travel_insurance_2024.txt


# Agent: Conversation Transcript Analyst (Travel Insurance)

```markdown
# Policy Summary

The travel insurance policy offers comprehensive coverage for individuals and families, providing worldwide 24/7 protection. The policy includes multiple coverage options with varying premiums for different regions and travel durations. Key benefits include medical and additional expenses, emergency evacuation, luggage and personal effects, trip cancellations, and more. Exclusions and specific conditions apply, especially for pre-existing conditions and certain activities.

## Age

- **Customer Age: 62**
- **Spouse Age: 60**

### Underwriting Considerations:
- Customers under 69 will be considered under the "Adult maximum 69 years old" category for benefit limits.
- **Medical and Additional Expenses**: Up to IDR 2,500,000,000 (Platinum plan) for customers under 70.
- **Premium Examples**: Starts from IDR 200 (Platinum, Worldwide, 1-4 days).

## Medical Conditions

- **Conditions**: High blood pressure, diabetes (under control with medication, no hospitalization in past 12 months).

### Underwriting Considerations:
- Existence of these conditions typically necessitates assessment for additional premiums or exclusions.
- The policy excludes claims arising within the first year of onset if classified as pre-existing conditions.
- Ensure medications for chronic conditions do not fall outside standard travel healthcare provisions.

## Destination

- **Destination**: Japan

### Underwriting Considerations:
- Coverage must include areas in Asia Pacific, with Platinum, Gold, and Silver options.
- Pay attention to COVID-19 specific coverage which provides additional benefits relevant to Japan entry requirements.

## Coverage Factors

- **Medical Emergencies**: Desired coverage includes emergency evacuation, hospital daily cash benefit, outpatient treatment.
- **Other Concerns**: Flight delay, luggage loss, and trip cancellation.

### Underwriting Considerations:
- Ensure sufficient limits for medical evacuation (Unlimited across all plans).
- Daily hospital benefit for hospitalization due to any insured events, including COVID-19.
- Confirm reimbursement capabilities in case of trip delay/cancellation up to set limits (e.g., IDR 65,000,000 for Platinum).

## Policy Customization Factors

- **Customization**: Preference for lower excess on premium plan.

### Considerations:
- Policy customization should allow limits and deductibles to reflect the customer's willingness to pay higher premiums for reduced personal payouts in claims scenarios.
- Advocate for policy benefits enhancement suitable for aged travelers seeking greater peace of mind about personal health contingencies while abroad.

---

**Transcripts Examples**: Customer expressed interest in light hiking and cultural experiences like visiting Mount Fuji and onsens. It's crucial these activities align with the recreational activities covered under the policy.

**Conclusion**: By assessing the specific needs and preferences of the customer alongside policy benefits, appropriate travel insurance options can be structured, potentially reducing risks and ensuring adequate protection during their trip. Further discussion with the customer may fine-tune the offering, particularly regarding comprehensive medical and critical incident coverage.
``` 

This Markdown report offers structured insights and considerations for the Chief Underwriter based on the provided customer context and policy details.

Notes to self: 
- More explicit referencing of insurance policy. Defined in agent or task?
- IDR?
- Table comparison format?

In [11]:
print(recsys_crew.usage_metrics)

total_tokens=42312 prompt_tokens=39990 completion_tokens=2322 successful_requests=15
